### The project is organized by the following steps:
1. Analyze data before matching
2. Match data
3. Analyze data after matching

### Analyze data before matching

In [4]:
# import packages for completing steps
import pandas as pd
import numpy as np
from tableone import TableOne
import math
import matplotlib.pyplot as plt
import cv2

In [5]:
df = pd.read_csv('simulated_data.csv')
df=df.drop('Unnamed: 0',axis=1)
df.head()

,id,year,Dept,treatment,profit,size,wageindex
0,110003.0,2003,M5,0,4926.0,164.0,0.675814
1,110003.0,2004,M5,0,5194.0,141.0,0.820107
2,110003.0,2005,M5,0,5347.0,152.0,1.194434
3,110003.0,2006,M5,0,5502.0,152.0,0.632703
4,110003.0,2007,M5,0,5814.0,126.0,0.932007


In [6]:
# check the properties of the dataset
print(df.shape)
print(df.dtypes)

(29972, 7)
id           float64
year           int64
Dept          object
treatment      int64
profit       float64
size         float64
wageindex    float64
dtype: object


In [7]:
# check missing values
df.isnull().sum()

id           30
year          0
Dept          0
treatment     0
profit        0
size          0
wageindex     0
dtype: int64

In [8]:
df[df['id'].isnull()].head(1)

,id,year,Dept,treatment,profit,size,wageindex
14610,NaN,2003,M19,0,4786.0,126.0,1.12994


In [9]:
print(df[(df['treatment']==1) & (df['Dept']=='O11')].shape)
print(df[(df['treatment']==0) & (df['Dept']=='O11')].shape)

print(df[(df['treatment']==1) & (df['Dept']=='M19')].shape)
print(df[(df['treatment']==0) & (df['Dept']=='M19')].shape)

(11, 7)
(743, 7)
(7, 7)
(630, 7)


Because the number of control group is far greater than number of treated group. Therefore, we can delete these missing values since they are all in control groups.

In [10]:
df=df.dropna()

In [11]:
df['id']=df['id'].astype(int)
df['profit']=df['profit'].astype(int)

In [12]:
xvars = df.drop('treatment',axis=1)
xvars_names1 = [name for name in df.columns if name not in['profit','size','wageindex']]
xvars_names = [name for name in df.columns if name in['profit','size','wageindex']]
xvars_names1

['id', 'year', 'Dept', 'treatment']

In [13]:
df1 = df.drop(['id', 'year', 'Dept'],1)
df1.head()
df1['treatment']=df1['treatment'].astype(str)

#### We use R packages Tableone to analyze before matching, because R could give us a direct and obvious picture. The greedy match process will still be conducted in Jupyter notebook, since Python is faster. The above is the picture of the result in R.

In [14]:
src=cv2.imread('Analyze before matching.png')       
cv2.namedWindow('input_image', cv2.WINDOW_AUTOSIZE)
cv2.imshow('input_image', src)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>

#### We can see that the SMD of profit is 0.303,greater than 0.2, meaning serious imbalance. The smd of size and wageindex are 0.198,0.112 respectively. If the smd is between 0.1 and 0.2, this means that values are not too alarming. 

### Match treated group with control group

In [15]:
treated = df[df['treatment']==1]
control = df[df['treatment']==0]
created = treated.reset_index(drop=True)
control = control.reset_index()
m = control.head()
n = 2
m = m.drop(m.index[n])

### find in which department, the number of treat groups exceeds the control group.

In [16]:
n=0
for unique_dept in treated.Dept.unique().tolist():
    unique_dep_df = treated[treated['Dept']==unique_dept]
    #treat_len = len(unique_dep_df)
    unique_dep_d = control[control['Dept']==unique_dept]
    #contro_len = len(unique_dep_df1) 
    if len(unique_dep_df) > len(unique_dep_d):
        n +=1
        #print(unique_dept)
print(n)
    
    

0


We find that in all departments, the number of control groups exceed the number of treat group.

In [17]:
m = treated.head(4).reset_index()
#m.drop('index')
m.drop(['id','year'],1)

,index,Dept,treatment,profit,size,wageindex
0,23768,M5,1,5347,162.0,0.866871
1,23783,M5,1,5101,131.0,1.196583
2,23792,M1,1,8886,372.0,0.755190
3,23807,M1,1,24182,347.0,0.864659


In [18]:

treated = df[df['treatment']==1]
control = df[df['treatment']==0]
created = treated.reset_index(drop=True)
control = control.reset_index(drop=True)
n=0
temp = control
final = pd.DataFrame()
control1 = control

# we tranverse all rows in treated dataset. When we find match, we delete from the control dataset.
for i in range(0,len(treated)):
    # We create a list called dis to store the values which measure the distance between one treated group and all control group.
    dis = []
    inde=[]
    
    for j in range(0,len(control)):     
        # We ignore 'math domain error',because the distance of error row is much larger than ohters.
        try:
            # To meet the condition,only people in the same department can be matched.
            if treated.iloc[i,2]==control.iloc[j,2]:
                distance1 = treated.iloc[i,4]-control.iloc[j,4]
                distance2 = treated.iloc[i,5]-control.iloc[j,5]
                distance3 = treated.iloc[i,6]-control.iloc[j,6]
                distance_square = distance1**2 + distance2**2+distance3**2
                distance = math.sqrt(distance_square)
                dis.append(distance)
            else:
                continue
        except ValueError:
            # find the error row and delete in later steps.
            inde.append(j)
        #print(inde)
    
    # create a temporary dataframe to merge with dis list.
    control_temp = control[control.Dept==treated.iloc[i,2]]
    
    
    if inde != []:
        control_temp = control_temp.drop(control.index[list(set(inde))])      
    
    control_temp['distance']=dis
    
    # select the shortest distance
    control_temp = control_temp.sort_values('distance').reset_index()
    temp1 = control_temp.sort_values('distance').reset_index().head(1)
    
    # find the original index in control dataset.
    orignial_idex = temp1.iloc[0,1]
    temp2 = temp1.append(treated.iloc[[i]])
    temp2['match_id']=n
    n = n+1
    final = final.append(temp2)
    
    # delete the row which has been matched
    control = control.drop(control.index[orignial_idex]).reset_index(drop=True)
    
    
    print(len(treated))
    print(n)
    
    
   

    



C:\Users\Yidan Wang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yidan Wang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


436
1
436
2
436
3
436
4
436
5
436
6
436
7
436
8
436
9
436
10
436
11
436
12
436
13
436
14
436
15
436
16
436
17
436
18
436
19
436
20
436
21
436
22
436
23
436
24
436
25
436
26
436
27
436
28
436
29
436
30
436
31
436
32
436
33
436
34
436
35
436
36
436
37
436
38
436
39
436
40
436
41
436
42
436
43
436
44
436
45
436
46
436
47
436
48
436
49
436
50
436
51
436
52
436
53
436
54
436
55
436
56
436
57
436
58
436
59
436
60
436
61
436
62
436
63
436
64
436
65
436
66
436
67
436
68
436
69
436
70
436
71
436
72
436
73
436
74
436
75
436
76
436
77
436
78
436
79
436
80
436
81
436
82
436
83
436
84
436
85
436
86
436
87
436
88
436
89
436
90
436
91
436
92
436
93
436
94
436
95
436
96
436
97
436
98
436
99
436
100
436
101
436
102
436
103
436
104
436
105
436
106
436
107
436
108
436
109
436
110
436
111
436
112
436
113
436
114
436
115
436
116
436
117
436
118
436
119
436
120
436
121
436
122
436
123
436
124
436
125
436
126
436
127
436
128
436
129
436
130
436
131
436
132
436
133
436
134
436
135
436
136
436
137
436
138
436


In [3]:
final1 = final
final = final.drop(['index','level_0'],1).reset_index(drop=True)

NameError: name 'final' is not defined

In [48]:
final = final
final.head()


,Dept,distance,id,profit,size,treatment,wageindex,year,match_id
0,M5,10.000231,110070,5357,162.0,0,0.934819,2013,0
1,M5,NaN,110070,5347,162.0,1,0.866871,2015,0
2,M5,34.438550,110610,5132,116.0,0,1.079371,2006,1
3,M5,NaN,110505,5101,131.0,1,1.196583,2015,1
4,M1,210.053613,141395,8825,171.0,0,1.476467,2016,2


### Analyze after matching

In [76]:
final.to_csv("final.csv", index=False, header=True)

In [ ]:
src=cv2.imread('Analyze after matching.png')       
cv2.namedWindow('input_image', cv2.WINDOW_AUTOSIZE)
cv2.imshow('input_image', src)
cv2.waitKey(0)
cv2.destroyAllWindows

The smd of profit is below 0.001. The smd of size and wageindex is 0.079. This result means that our match is reasonable. 